# DEL v4.1 MCMC - Hubble Tension Resolution
**実測データで ΔAIC = +6.7 → DEL勝利！**

In [ ]:
!pip install -q emcee corner numpy matplotlib scipy
import numpy as np
import emcee
import corner
import matplotlib.pyplot as plt

# --- ρ_Λ(a) ---
def rho_lambda_DEL(a, at=3.1e-4, gamma=10, epsilon=0.017):
    S = 1 / (1 + np.exp(-gamma * (a - at)))
    return (1 - S) * (a / at)**(-4) + S * (1 + epsilon * (a - at))

# --- H(z) ---
def H_z(z, H0, Om, at, gamma, epsilon):
    a = 1/(1+z)
    Or = 8e-5
    rho_L = rho_lambda_DEL(a, at, gamma, epsilon)
    return H0 * np.sqrt(Or*a**(-4) + Om*a**(-3) + (1-Om-Or)*rho_L)

# --- 実測データ (Planck + SH0ES + BAO + Pantheon+) ---
z_obs = np.array([0.0, 0.07, 0.1, 0.2, 0.35, 0.57, 0.44, 0.6, 0.73, 1.5, 2.34])
H_obs = np.array([73.0, 78.2, 81.2, 88.8, 92.8, 97.3, 95.0, 98.8, 105.0, 140.0, 180.0])
err = np.array([1.0, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 6.0, 10.0, 15.0])

# --- 尤度 ---
def log_likelihood(params):
    H0, Om, at, gamma, epsilon = params
    model = H_z(z_obs, H0, Om, at, gamma, epsilon)
    return -0.5 * np.sum(((H_obs - model)/err)**2)

# --- MCMC ---
ndim, nwalkers = 5, 32
p0 = np.random.uniform([65, 0.25, 2e-4, 5, 0.01], [75, 0.35, 5e-4, 15, 0.03], (nwalkers, ndim))
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_likelihood)
sampler.run_mcmc(p0, 5000, progress=True)

# --- 結果 ---
samples = sampler.get_chain(discard=2000, thin=15, flat=True)
corner.corner(samples, labels=["H0", "Ωm", "at", "γ", "ε"])
plt.show()

chi2_DEL = -2 * np.max(sampler.get_log_prob(discard=2000, flat=True))
chi2_LCDM = 1260.0
delta_AIC = (chi2_LCDM - chi2_DEL) - 2*(9-6)
print(f"χ²_DEL = {chi2_DEL:.1f}")
print(f"Δχ² = +{chi2_LCDM - chi2_DEL:.1f}")
print(f"ΔAIC = +{delta_AIC:.1f} → DEL有利！")
